In [ ]:
# Cell 1 - UPDATED FOR YOUR SPECIFIC PATH
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Your specific path - train.csv is now in data/raw/
train_path = 'data/raw/train.csv'

# Check if file exists
if os.path.exists(train_path):
    # Load the data
    train_df = pd.read_csv(train_path)
    
    print("✅ Data loaded successfully!")
    print(f"File: {train_path}")
    print(f"Size: {os.path.getsize(train_path)/1024/1024:.2f} MB")
    print(f"Shape: {train_df.shape}")
    print(f"Columns: {list(train_df.columns)}")
    
    # Check if we have a test file
    test_path = 'data/raw/test.csv'
    if os.path.exists(test_path):
        test_df = pd.read_csv(test_path)
        print(f"\n✅ Test data loaded: {test_df.shape}")
    else:
        print(f"\n⚠️ No test.csv found, using train as reference")
        test_df = train_df.copy()
        
else:
    print(f"❌ ERROR: File not found at {train_path}")
    print("Current directory:", os.getcwd())
    print("\nFiles in data/raw/:")
    for f in os.listdir('data/raw'):
        print(f"  - {f}")
    
    # Create empty dataframes to prevent further errors
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()

❌ ERROR: File not found at data/raw/train.csv
Current directory: c:\Projects\credit-risk-model\notebooks

Files in data/raw/:


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/raw'

In [ ]:
print("=" * 80)
print("BASIC DATA OVERVIEW")
print("=" * 80)

# Display first few rows
train_df.head()

# Check data types
print("\n📊 DATA TYPES:")
print(train_df.dtypes)

# Check for missing values
print("\n🔍 MISSING VALUES:")
missing = train_df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✅ No missing values found!")

# Basic statistics
print("\n📈 BASIC STATISTICS:")
print(train_df.describe())

In [ ]:
# Create subplots for visualizations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Key Feature Distributions', fontsize=16)

# 1. Transaction Amount Distribution
axes[0, 0].hist(train_df['Amount'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Transaction Amount Distribution')
axes[0, 0].set_xlabel('Amount')
axes[0, 0].set_ylabel('Frequency')

# 2. Fraud Distribution
fraud_counts = train_df['FraudResult'].value_counts()
axes[0, 1].bar(fraud_counts.index, fraud_counts.values, color=['green', 'red'])
axes[0, 1].set_title('Fraud Distribution (0=No, 1=Yes)')
axes[0, 1].set_xlabel('Fraud Result')
axes[0, 1].set_ylabel('Count')
for i, v in enumerate(fraud_counts.values):
    axes[0, 1].text(i, v, str(v), ha='center', va='bottom')

# 3. Product Category Distribution
top_categories = train_df['ProductCategory'].value_counts().head(10)
axes[0, 2].barh(range(len(top_categories)), top_categories.values)
axes[0, 2].set_yticks(range(len(top_categories)))
axes[0, 2].set_yticklabels(top_categories.index)
axes[0, 2].set_title('Top 10 Product Categories')
axes[0, 2].set_xlabel('Count')

# 4. Channel Distribution
channel_counts = train_df['ChannelId'].value_counts()
axes[1, 0].pie(channel_counts.values, labels=channel_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('Transaction Channels')

# 5. Transaction Hour Distribution
train_df['hour'] = pd.to_datetime(train_df['TransactionStartTime']).dt.hour
hour_counts = train_df['hour'].value_counts().sort_index()
axes[1, 1].plot(hour_counts.index, hour_counts.values, marker='o')
axes[1, 1].set_title('Transactions by Hour of Day')
axes[1, 1].set_xlabel('Hour (0-23)')
axes[1, 1].set_ylabel('Transaction Count')
axes[1, 1].grid(True, alpha=0.3)

# 6. Country Code Distribution
country_counts = train_df['CountryCode'].value_counts().head(10)
axes[1, 2].bar(range(len(country_counts)), country_counts.values)
axes[1, 2].set_xticks(range(len(country_counts)))
axes[1, 2].set_xticklabels(country_counts.index, rotation=45)
axes[1, 2].set_title('Top 10 Country Codes')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 80)
print("CUSTOMER-LEVEL ANALYSIS")
print("=" * 80)

# Calculate basic customer metrics
customer_stats = train_df.groupby('CustomerId').agg({
    'TransactionId': 'count',
    'Amount': ['sum', 'mean', 'std'],
    'FraudResult': 'sum'
})

customer_stats.columns = ['transaction_count', 'total_amount', 'avg_amount', 'std_amount', 'fraud_count']
customer_stats = customer_stats.reset_index()

print(f"📊 Unique customers: {customer_stats.shape[0]:,}")
print(f"📊 Average transactions per customer: {customer_stats['transaction_count'].mean():.2f}")
print(f"📊 Customers with fraud: {(customer_stats['fraud_count'] > 0).sum():,}")
print(f"📊 Fraud rate among customers: {(customer_stats['fraud_count'] > 0).mean()*100:.2f}%")

# Display customer statistics
print("\n📈 Customer Statistics Summary:")
print(customer_stats.describe())

# Plot customer transaction distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Transaction count distribution
axes[0].hist(customer_stats['transaction_count'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Transactions per Customer')
axes[0].set_xlabel('Number of Transactions')
axes[0].set_ylabel('Number of Customers')

# Total amount distribution
axes[1].hist(customer_stats['total_amount'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Total Amount per Customer')
axes[1].set_xlabel('Total Amount')
axes[1].set_ylabel('Number of Customers')

# Average amount distribution
axes[2].hist(customer_stats['avg_amount'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[2].set_title('Average Transaction Amount per Customer')
axes[2].set_xlabel('Average Amount')
axes[2].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 80)
print("TIME-BASED ANALYSIS")
print("=" * 80)

# Convert to datetime
train_df['TransactionStartTime'] = pd.to_datetime(train_df['TransactionStartTime'])

# Extract time features
train_df['date'] = train_df['TransactionStartTime'].dt.date
train_df['day_of_week'] = train_df['TransactionStartTime'].dt.day_name()
train_df['month'] = train_df['TransactionStartTime'].dt.month
train_df['hour'] = train_df['TransactionStartTime'].dt.hour

# Daily transaction trends
daily_transactions = train_df.groupby('date').agg({
    'TransactionId': 'count',
    'Amount': 'sum',
    'FraudResult': 'sum'
}).reset_index()

daily_transactions.columns = ['date', 'transaction_count', 'total_amount', 'fraud_count']

print("📅 Daily Transaction Trends:")
print(f"Date range: {daily_transactions['date'].min()} to {daily_transactions['date'].max()}")
print(f"Total days: {daily_transactions.shape[0]}")

# Plot daily trends
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Daily transaction count
axes[0, 0].plot(daily_transactions['date'], daily_transactions['transaction_count'])
axes[0, 0].set_title('Daily Transaction Count')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Transactions')
axes[0, 0].tick_params(axis='x', rotation=45)

# Daily total amount
axes[0, 1].plot(daily_transactions['date'], daily_transactions['total_amount'])
axes[0, 1].set_title('Daily Total Amount')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Total Amount')
axes[0, 1].tick_params(axis='x', rotation=45)

# Day of week analysis
dow_counts = train_df['day_of_week'].value_counts()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = dow_counts.reindex(dow_order)
axes[1, 0].bar(dow_counts.index, dow_counts.values)
axes[1, 0].set_title('Transactions by Day of Week')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Transaction Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# Hour of day analysis (already calculated)
hour_counts = train_df['hour'].value_counts().sort_index()
axes[1, 1].bar(hour_counts.index, hour_counts.values)
axes[1, 1].set_title('Transactions by Hour of Day')
axes[1, 1].set_xlabel('Hour (0-23)')
axes[1, 1].set_ylabel('Transaction Count')

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 80)
print("CORRELATION ANALYSIS")
print("=" * 80)

# Select numerical columns for correlation
numerical_cols = ['Amount', 'Value', 'FraudResult', 'hour']
corr_data = train_df[numerical_cols].copy()

# Calculate correlation matrix
correlation_matrix = corr_data.corr()

print("📊 Correlation Matrix:")
print(correlation_matrix)

# Visualize correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix')
plt.show()

# Check correlation with FraudResult
print("\n🔍 Correlation with FraudResult:")
fraud_corr = correlation_matrix['FraudResult'].sort_values(ascending=False)
for feature, corr_value in fraud_corr.items():
    if feature != 'FraudResult':
        print(f"{feature}: {corr_value:.4f}")

In [ ]:
print("=" * 80)
print("OUTLIER DETECTION")
print("=" * 80)

# Create box plots for numerical features
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Amount box plot
axes[0].boxplot(train_df['Amount'].dropna())
axes[0].set_title('Transaction Amount Box Plot')
axes[0].set_ylabel('Amount')

# Value box plot
axes[1].boxplot(train_df['Value'].dropna())
axes[1].set_title('Transaction Value Box Plot')
axes[1].set_ylabel('Value')

# Transaction count per customer box plot
axes[2].boxplot(customer_stats['transaction_count'])
axes[2].set_title('Transactions per Customer Box Plot')
axes[2].set_ylabel('Transaction Count')

plt.tight_layout()
plt.show()

# Calculate outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("\n📊 Outlier Analysis for Amount:")
amount_outliers, lower_amt, upper_amt = detect_outliers_iqr(train_df, 'Amount')
print(f"Lower bound: {lower_amt:.2f}, Upper bound: {upper_amt:.2f}")
print(f"Number of outliers: {len(amount_outliers):,} ({len(amount_outliers)/len(train_df)*100:.2f}%)")

print("\n📊 Outlier Analysis for Customer Transaction Count:")
count_outliers, lower_cnt, upper_cnt = detect_outliers_iqr(customer_stats, 'transaction_count')
print(f"Lower bound: {lower_cnt:.2f}, Upper bound: {upper_cnt:.2f}")
print(f"Number of outlier customers: {len(count_outliers):,} ({len(count_outliers)/len(customer_stats)*100:.2f}%)")

In [ ]:
print("=" * 80)
print("SUMMARY OF KEY INSIGHTS")
print("=" * 80)

# Compile key insights
insights = []

# Insight 1: Data Size
insights.append(f"1. Dataset contains {train_df.shape[0]:,} transactions for {customer_stats.shape[0]:,} unique customers.")

# Insight 2: Fraud Rate
fraud_rate = train_df['FraudResult'].mean() * 100
insights.append(f"2. Overall fraud rate is {fraud_rate:.4f}% ({train_df['FraudResult'].sum():,} fraudulent transactions).")

# Insight 3: Customer Behavior
avg_transactions = customer_stats['transaction_count'].mean()
insights.append(f"3. Average customer has {avg_transactions:.1f} transactions (range: {customer_stats['transaction_count'].min()} to {customer_stats['transaction_count'].max()}).")

# Insight 4: Time Patterns
busiest_hour = train_df['hour'].value_counts().idxmax()
insights.append(f"4. Peak transaction hour: {busiest_hour}:00 with {train_df['hour'].value_counts().max():,} transactions.")

# Insight 5: Monetary Patterns
top_category = train_df['ProductCategory'].value_counts().index[0]
top_category_pct = train_df['ProductCategory'].value_counts().iloc[0] / len(train_df) * 100
insights.append(f"5. Most popular product category: '{top_category}' ({top_category_pct:.1f}% of transactions).")

# Print insights
print("🔍 TOP 5 INSIGHTS FOR FEATURE ENGINEERING:")
for i, insight in enumerate(insights, 1):
    print(f"{insight}")

# Save insights to file
with open('../notebooks/eda_insights.txt', 'w') as f:
    f.write("EDA KEY INSIGHTS\n")
    f.write("================\n\n")
    for insight in insights:
        f.write(f"{insight}\n")
    f.write(f"\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n💾 Insights saved to: notebooks/eda_insights.txt")

# Display sample of processed data
print("\n📋 SAMPLE OF ENRICHED DATA:")
sample_cols = ['CustomerId', 'TransactionStartTime', 'Amount', 'ProductCategory', 'ChannelId', 'hour', 'day_of_week']
print(train_df[sample_cols].head(10))